In [1]:
import pathlib
import numpy as np, matplotlib.pyplot as plt
from matplotlib.patches import FancyArrow
from PIL import Image
from flyerdetection.flyer_detection import Flyer_Detection

In [42]:
root = pathlib.Path().resolve().parent.parent
#camera_dir_name = "Camera_10_35_34"
#camera_dir_name = "Camera_11_30_10"
#camera_dir_name = "Camera_11_33_43"
#camera_dir_name = "Camera_11_38_00"
#camera_dir_name = "Camera_11_40_23"
#camera_dir_name = "Camera_11_43_33"
#camera_dir_name = "Camera_11_45_59"
#camera_dir_name = "Camera_11_48_16"
#camera_dir_name = "Camera_11_49_43"
#camera_dir_name = "Camera_12_01_40"
camera_dir_name = "Camera_17_15_49"
original_ims_path = root / "flyer_image_examples" / camera_dir_name
analysis_ims_path = root / "OUTPUT" / camera_dir_name

In [43]:
temp = Flyer_Detection()
temp.create_df_from_input_location(str(original_ims_path), str(analysis_ims_path.parent))

In [44]:
analyzer = Flyer_Detection()

gif_frames_path = root / "OUTPUT" / f"{camera_dir_name}_gif_frames"
if not gif_frames_path.is_dir():
    gif_frames_path.mkdir()

for original_im_path in original_ims_path.glob("*.bmp"):
    frame_number = int(original_im_path.stem.split("_")[-1])
    if frame_number not in range(90, 161):
        continue
    original_im_arr = np.asarray(Image.open(original_im_path))
    analysis_im_path = analysis_ims_path / original_im_path.name
    if not analysis_im_path.is_file():
        f, ax = plt.subplots()
        ax.imshow(original_im_arr, cmap="gray")
        ax.axis("off")
        plt.savefig(
            gif_frames_path / original_im_path.name.replace(".bmp", ".png"),
            bbox_inches="tight",
        )
        plt.close()
        continue
    analysis_im_arr_as_read = np.asarray(Image.open(analysis_im_path))
    analysis_im_arr = np.zeros_like(original_im_arr)
    analysis_im_arr[: analysis_im_arr_as_read.shape[0], :] = analysis_im_arr_as_read
    filtered_image = analyzer.filter_image(original_im_arr)
    result = analyzer.radius_from_lslm(
        filtered_image,
        original_im_path.relative_to(root),
        root / "OUTPUT",
        min_radius=0,
        max_radius=np.inf,
        save_output_file=False,
    )
    arrow = FancyArrow(
        np.median(result.flyer_column),
        np.median(result.flyer_row),
        0.75 * (result.center_column - np.mean(result.flyer_column)),
        0.75 * (result.center_row - np.mean(result.flyer_row)),
        width=0.1,
        color="r",
    )
    f, ax = plt.subplots()
    ax.imshow(original_im_arr, cmap="gray")
    analysis_im_alphas = 0.5 * np.ones_like(analysis_im_arr)
    analysis_im_alphas[analysis_im_arr == 0] = 0
    ax.imshow(analysis_im_arr, alpha=analysis_im_alphas)
    ax.add_patch(arrow)
    ax.axis("off")
    plt.savefig(
        gif_frames_path / original_im_path.name.replace(".bmp", ".png"),
        bbox_inches="tight",
    )
    plt.close()


In [45]:
all_frames_paths = [frame_path for frame_path in gif_frames_path.glob("*.png")]
all_frames_paths.sort(key=lambda x: int(x.stem.split("_")[-1]))
frames = [Image.open(image_fp) for image_fp in all_frames_paths]
frame_one = frames[0]
frame_one.save(
    analysis_ims_path.parent/f"{camera_dir_name}_flyer_analysis.gif",
    format="GIF",
    append_images=frames,
    save_all=True,
    duration=100,
    loop=1,
)
